In [1]:
import requests
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page

<Response [200]>

Web scrapping the table from the Wikipedia site.

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'lxml')

In [3]:
table = soup.find('div', class_ = 'mw-parser-output').table

Create pandas dataframe with the data from the table of wikipedia page.

In [4]:
data = []

In [5]:
columns = ['PostalCode', 'Borough', 'Neighbourhood']

In [6]:
for tr in table.find_all('tr'):
    try:
        templist = []
        i = 0
        for td in tr.find_all('td'):
            templist.append(td.text.strip())
            i += 1
            if i == 3:
                data.append(templist)
                del templist
                i = 0
    except Exception as e:
        pass
    

In [7]:
import pandas as pd
df = pd.DataFrame(data, columns = columns)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop all the cells with *Borough* that is **Not assigned**.

In [8]:
df[df['Borough'] == 'Not assigned'].index

Int64Index([  0,   1,   9,  13,  20,  21,  30,  36,  37,  45,  46,  50,  51,
             52,  54,  55,  59,  60,  61,  73,  74,  75,  88,  89,  90, 104,
            105, 106, 120, 121, 136, 137, 148, 149, 155, 161, 162, 167, 175,
            181, 182, 188, 189, 190, 194, 195, 201, 202, 203, 204, 209, 210,
            223, 224, 237, 238, 241, 242, 247, 248, 253, 254, 258, 259, 260,
            261, 263, 264, 274, 275, 276, 277, 278, 279, 280, 281, 287],
           dtype='int64')

In [9]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True)
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Replace *Neighbourhood* to be same as the borough if the cell has **Not assigned** neighbourhood.

In [10]:
ind = df[df['Neighbourhood'] == 'Not assigned'].index

In [11]:
df.loc[ind,'Borough'].values

array(["Queen's Park"], dtype=object)

In [12]:
df.loc[ind,'Neighbourhood'] = df.loc[ind,'Borough'].values
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Combine *Neighbourhood* separated with comma for **repeated** postal code area

In [13]:
df2 = df[df['PostalCode'].duplicated() == True]
df2.head()

,PostalCode,Borough,Neighbourhood
5,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Manor
12,M1B,Scarborough,Malvern
16,M4B,East York,Parkview Hill
18,M5B,Downtown Toronto,Garden District


In [14]:
df.drop(df2.index, inplace = True)
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
8,M7A,Queen's Park,Queen's Park


In [15]:
for index, data2 in df2.iterrows():
    for index, data in df.iterrows():
        if data2['PostalCode'] == data['PostalCode']:
            data['Neighbourhood'] = data['Neighbourhood'] + ', ' + data2['Neighbourhood']

In [16]:
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park


In [17]:
df.shape

(103, 3)

Merge the dataframe with *geographical coordinates* for each postal code

In [19]:
dfx = pd.read_csv('Geospatial_Coordinates.csv')
dfx.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
dfx.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)

In [21]:
df_new = pd.merge(df, dfx, on = 'PostalCode')

In [22]:
df_new.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
